In [111]:
%%capture
!pip install matatika==1.17.0
!pip install pandas==1.4.3 numpy==1.26.4
!pip install weasyprint==58.1
!pip install matatika-iplotter==1.3.0
!pip install git+https://github.com/Matatika/htmlcharts.git@v0.1.0

In [112]:
import pandas as pd
import os
from datetime import datetime, timedelta, date
from htmlcharts.bar import bar_chart
from matatika.library import MatatikaClient
from matatika.types import DataFormat, Resource
from iplotter import ChartJSPlotter
from weasyprint import HTML, CSS

In [ ]:
from matatika.library import MatatikaClient

endpoint_url = os.getenv('ENDPOINT_URL')
workspace_id = os.getenv('WORKSPACE_ID')

# Attempt to use the pipeline 'AUTH_TOKEN'
auth_token = os.getenv('AUTH_TOKEN')
# On 401 use CLIENT_ID and CLIENT_SECRET to refresh
client_id = os.getenv('NOTEBOOK_CLIENT_ID')
client_secret = os.getenv('NOTEBOOK_CLIENT_SECRET')

client = MatatikaClient(auth_token=auth_token, client_id=client_id, client_secret=client_secret, endpoint_url=endpoint_url, workspace_id=workspace_id)
plotter = ChartJSPlotter()

In [114]:
expensive_queries = client.fetch("snowflake-monitoring/dashboard/top_10_costing_queries_last_7_days")
expensive_queries_df = pd.DataFrame(expensive_queries)


# Chart data
chart_dataset_id = 'snowflake-monitoring/dashboard/snowflake_spend_last_7_days'

dataset_information = client.get_dataset(chart_dataset_id)

chart_data = client.fetch(chart_dataset_id)

In [115]:
# Dates logic
current_date = datetime.now()
seven_days_ago_date = datetime.now() - timedelta(days=6)

if current_date not in chart_data:
    current_date = current_date - timedelta(days=1)
    current_date = current_date - timedelta(days=7)

current_date_str = current_date.strftime("%B %d, %Y")
seven_days_ago_str = seven_days_ago_date.strftime("%B %d, %Y")

date_range = f"{seven_days_ago_str} - {current_date_str}"

last_seven_days_dates = []

for i in range(6, -1, -1):
    date_str = (current_date - timedelta(days=i)).strftime("%Y-%m-%d")
    last_seven_days_dates.append(date_str)

In [116]:
credits_burndown = client.fetch("snowflake-monitoring/dashboard/snowflake_daily_burndown")

credits_remaining = client.fetch("snowflake-monitoring/dashboard/snowflake_credits")


daily_credits_usage = credits_burndown[0]['snowflake_spend.daily']
burndown_days_remaining = credits_burndown[0]['snowflake_spend.burndown']

overall_credits_remaining = credits_remaining[0]['Snowflake Credits']

In [117]:
# Set up email html

html = """
<head>
    <style>
        .content {
            max-width: 1000px;
            text-align: left;
            margin: auto;
        }

        .bar-display {
            font: 18px sans-serif;
            text-align: center;
            color: rgb(120, 120, 120);
            position: relative;
            margin-top: auto;
        }

        .label {
            display: block;
            color: black;
            font-size: 14px;
        }

        .flex-charts {
            align-items: center; 
            justify-content: center;
        }

        .m2 {
            margin-bottom: 2px;
            margin-top: 2px;
        }

        .mb5 {
            margin-bottom: 5px;
        }

        .mb15 {
            margin-bottom: 15px;
        }

        .mb30 {
            margin-bottom: 30px;
        }

        .mt0 {
            margin-top: 0px;
        }

        .mt10 {
            margin-top: 10px;
        }

        .mt15 {
            margin-top: 15px;
        }

        .mt30 {
            margin-top: 30px;
        }

        .pb10 {
            padding-bottom: 10px;
        }

        .pl10 {
            padding-left: 10px;
        }

        .pr10 {
            padding-right: 10px;
        }

        hr {
            border: 0;
            height: 1px;
            border-top: 1px solid rgba(0, 0, 0, 0.2);
            border-bottom: 1px solid rgba(255, 255, 255, 0.2);
        }

        .dataframe td {
            overflow-wrap: anywhere;
        }

        .dataframe {
            table-layout: auto;
            width: 100%;
        }

        .dataframe td {
            text-align: left;
        }

    </style>
</head>
<div class="content">
"""

html += f'''
<div class="header">
<img src="https://app.matatika.com/assets/images/utility/matatika.png" style="width: 80px; float: left;"/>
<div style="text-align: right;">
<h1 class="mb5">Snowflake Weekly Cost Report</h1>
<p class="mt10">{date_range}</p>
</div>
</div>
<hr class="mb30 mt30">
<div>
<div class="pr10 pb10">
<h4 class="mt0 mb5">Avg. Usage per Day</h4>
<h2 class="m2 mb5">{daily_credits_usage}</h2>
<p class="m2 mb5">{burndown_days_remaining}</p>
<p class="m2">{overall_credits_remaining}</p>
</div>
</div>
<a href="https://app.matatika.com/l/{workspace_id}/dashboard" class="mt15 mb15">View Dashboard</a>
<h2 class="mb15" style="text-align: center;">Credits Used Per Day</h2>
<div class="flex-charts">
<div class="chart-holder">
'''

In [118]:
# Create chart for email

# Add data points for each day in the last 7 days missing from the data, set credits to 0
for d in last_seven_days_dates:
    dates_in_chart_data = []
    for data_point in chart_data:
        dates_in_chart_data.append(data_point["Date"])
    if d not in dates_in_chart_data:
        new_data_point = {}
        new_data_point['Date'] = d
        new_data_point['Credits Used'] = '0'
        chart_data.append(new_data_point)

data_dict = {}

# Make all credits values floats
for data_point in chart_data:
    data_dict[data_point['Date']] = float(data_point["Credits Used"])

# Order the dates
sorted_dict = dict(sorted(data_dict.items(), key=lambda x: x[0]))

chart_data_dict = {}

# Change the date keys into 'Mon', 'Tue'...
for date_str, value in sorted_dict.items():
    date = datetime.strptime(date_str, '%Y-%m-%d')
    chart_data_dict[date.strftime('%a')] = value
  
html += bar_chart(chart_data_dict, bar_color="rgba(255, 204, 203, 0.8)", chart_width=1000, bar_height=350)

html += '''
</div>
</div>
<hr class="mb30 mt30">
<div>
<h3>Top 10 Expensive Queries - Last 7 Days</h3>
'''

In [119]:
# Create tables to list queries in email
html_output = expensive_queries_df.to_html(index=False)

if expensive_queries:
    html += html_output.replace('style="text-align: right;"', 'border: 0;"').replace('border="1"', '')
else:
    html += '<p>No queries data...</p>'
html += '</div>'

In [120]:
# Write html out to file
with open("snowflake_cost_report_email_template.html", 'w') as f:
    f.write(html)